In [60]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from numpy.linalg import eig
import matplotlib.animation as animation
from IPython.display import clear_output
import subprocess
import random
from datetime import datetime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [61]:
def read_data(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()

    # Extracting layers
    layers = []
    for line in data:
        if line.strip() == 'Layers:':
            break
    for line in data[data.index('Layers:\n') + 1:]:
        if line.strip() == 'Materials:':
            break
        layer_info = line.strip().replace("(", "").replace(")", "").replace("'", "").split(', ')
        layer = (layer_info[0], float(layer_info[1]))
        layers.append(layer)

    # Extracting materials
    materials = {}
    for line in data[data.index('Materials:\n') + 1:]:
        if line.strip() == 'QWI Target Shift:':
            break
        material_info = line.strip().split(': ')
        material_name = material_info[0]
        properties = [float(prop) for prop in material_info[1][1:-1].split(', ')]
        materials[material_name] = properties

    # Extracting QWI target shift
    qwi_target_shift = float(data[data.index('QWI Target Shift:\n') + 1])

    # Extracting number of electric fields
    number_of_electric_fields = int(data[data.index('Number of Electric Fields:\n') + 1])

    # Extracting max applied electric field
    max_applied_electric_field = int(data[data.index('Max Applied Electric Field:\n') + 1])

    # Extracting FOM elements
    fom_elements = [float(data[data.index('FOM:\n') + i]) for i in range(1, 5)]

    return materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements


# Example usage:
# for files in file_path:
#     file_path = 'data.txt'  # Replace 'your_file_path_here.txt' with the actual file path
#     materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)
#     input_ = [materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements]
#     print(input_)
#     print(fom_elements)

In [62]:
import os
input_features_matrix=[]
figure_of_merit=[]
# Directory path
directory = 'Z:/FYP'

# Iterate over all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a text file
    if filename.endswith('.txt'):
        # Open the file
        file_path = os.path.join(directory, filename)
        materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)
        input_ = [materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements]
        input_features_matrix.append(input_)
        

In [63]:
input_features_matrix[1]

[{'GaAs': [0.111, 1.42, 0.063, 0.082, 0.51, 3.9476],
  'GaP': [-0.388, 2.74, 0.25, 0.14, 0.67, 3.3798],
  'InP': [0.0, 1.35, 0.077, 0.14, 0.6, 3.3688],
  'InAs': [0.441, 0.354, 0.023, 0.026, 0.41, 3.714],
  'AlAs': [-0.4245, 2.95, 0.15, 0.16, 0.79, 2.994],
  'In0.5314954Ga0.439968Al0.0285366As': [0.30811502529141294,
   0.7807065272131057,
   0.0442228682,
   0.0544621124,
   0.464840708,
   3.541396534932185],
  'In0.528113Ga0.238845Al0.233042As': [0.21488841394518937,
   1.0517929225205311,
   0.062150133999999996,
   0.070602948,
   0.52244046,
   3.5181736081437682],
  'In0.5279419999999999Ga0.228716Al0.243342As': [0.2093955292439803,
   1.0677652537249773,
   0.063053074,
   0.07141592399999999,
   0.52534156,
   3.516926308226299],
  'In0.5313672Ga0.432322Al0.0363108As': [0.3051258622467586,
   0.7893984813993644,
   0.044904351599999996,
   0.0550756792,
   0.467030304,
   3.5409168008977874],
  'In0.527935Ga0.228284Al0.243781As': [0.20915981970038872,
   1.0684506551311757,
   

In [64]:
import numpy as np
from sklearn.model_selection import train_test_split

# Initialize lists to store features and targets
X_train_list = []
y_train_list = []

# Iterate over each heterostructure
for heterostructure in input_features_matrix:
    # Extract features and targets from the current heterostructure
    materials = heterostructure[0]
    layers = heterostructure[1]
    qwi_target_shift = heterostructure[2]
    max_applied_electric_field = heterostructure[3]
    fom_elements = heterostructure[4]

    # Initialize list to store material features in the correct order
    material_features = []

    # Iterate over each layer to ensure correct material ordering
    for layer in layers:
        material_key = layer[0]
        if material_key in materials:
            material_features.extend(materials[material_key])

    # Flatten the layers list
    layer_thicknesses = [thickness for _, thickness in layers]

    # Combine all features into a single array
    all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

    # Append features to the features list
    X_train_list.append(all_features)

    # Append targets to the targets list
    y_train_list.append(fom_elements)
    
print(fom_elements)

# Print the shape of each element in X_train_list
# for i, x in enumerate(X_train_list):
#     print(f"Element {i}: Shape = {x.shape}")

# Convert the lists to numpy arrays
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training data shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nTesting data shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


[-0.001058095608190035, 0.10009279088660716, 2.1896345143733225, 0.20532481225338373]
Training data shapes:
X_train shape: (998, 23)
y_train shape: (998, 4)

Testing data shapes:
X_test shape: (250, 23)
y_test shape: (250, 4)


In [65]:
import numpy as np
from sklearn.model_selection import train_test_split

# Get the maximum shape among all elements in X_train_list
max_shape = max(x.shape[0] for x in X_train_list)

# Pad the elements to have the same shape
X_train_padded = np.array([np.pad(x, (0, max_shape - x.shape[0]), mode='constant') for x in X_train_list])

# Convert lists to numpy arrays
y_train = np.array(y_train_list)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_padded, y_train, test_size=0.2, random_state=42)

print("Training data shapes:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\nTesting data shapes:")
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


Training data shapes:
X_train shape: (998, 23)
y_train shape: (998, 4)

Testing data shapes:
X_test shape: (250, 23)
y_test shape: (250, 4)


In [66]:
import torch
import torch.nn as nn
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        #self.fc2 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Build neural network model
def build_model(input_size):
    model = NeuralNetwork(input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    return model, criterion, optimizer

# Example usage:
input_size = X_train.shape[1]  # Input size corresponds to the number of input features
model, criterion, optimizer = build_model(input_size)
print(model)


NeuralNetwork(
  (fc1): Linear(in_features=23, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)


In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import numpy as np

# Train the model
def train_model(model, criterion, optimizer, X_train, y_train, epochs=2000):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in zip(X_train, y_train):
            inputs = inputs.clone().detach().type(torch.float32)
            targets = targets.clone().detach().type(torch.float32)

            optimizer.zero_grad()

            outputs = model(inputs)
            targets = targets.view(-1)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(X_train)
        if epoch % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        inputs = X_test.clone().detach().type(torch.float32)
        targets = y_test.clone().detach().type(torch.float32)

        outputs = model(inputs)
        mse = mean_squared_error(targets.numpy(), outputs.numpy())
        print(f'Mean Squared Error (MSE) on test data: {mse:.4f}')

# Train and evaluate the model
def train_and_evaluate_model(model, criterion, optimizer, X_train, y_train, X_test, y_test):
    train_model(model, criterion, optimizer, X_train, y_train)
    evaluate_model(model, X_test, y_test)

# Convert input features and target values to numpy arrays
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)
X_test_np = np.array(X_test)
y_test_np = np.array(y_test)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)

# Example usage:
epochs = 2000
batch_size = 32
# Train and evaluate the model
train_and_evaluate_model(model, criterion, optimizer, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)

Epoch [1/2000], Loss: 6.0158
Epoch [11/2000], Loss: 0.9984
Epoch [21/2000], Loss: 0.8784
Epoch [31/2000], Loss: 0.8475
Epoch [41/2000], Loss: 0.8154
Epoch [51/2000], Loss: 0.8001
Epoch [61/2000], Loss: 0.7789
Epoch [71/2000], Loss: 0.7594
Epoch [81/2000], Loss: 0.7344
Epoch [91/2000], Loss: 0.7195
Epoch [101/2000], Loss: 0.6845
Epoch [111/2000], Loss: 0.6150
Epoch [121/2000], Loss: 0.5136
Epoch [131/2000], Loss: 0.4746
Epoch [141/2000], Loss: 0.4724
Epoch [151/2000], Loss: 0.4500
Epoch [161/2000], Loss: 0.4212
Epoch [171/2000], Loss: 0.4178
Epoch [181/2000], Loss: 0.4094
Epoch [191/2000], Loss: 0.4039
Epoch [201/2000], Loss: 0.3799
Epoch [211/2000], Loss: 0.3849
Epoch [221/2000], Loss: 0.4026
Epoch [231/2000], Loss: 0.3880
Epoch [241/2000], Loss: 0.3767
Epoch [251/2000], Loss: 0.3706
Epoch [261/2000], Loss: 0.3756
Epoch [271/2000], Loss: 0.3529
Epoch [281/2000], Loss: 0.3603
Epoch [291/2000], Loss: 0.3544
Epoch [301/2000], Loss: 0.3649
Epoch [311/2000], Loss: 0.3457
Epoch [321/2000], L

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [68]:
# Assuming you have a trained model named 'model'
file_path = f"Z:\FYP\data_2024-03-26_14-22-54.txt"

# Read the data from the file
materials, layers, qwi_target_shift, max_applied_electric_field, fom_elements = read_data(file_path)

# Extract material names from the 'Layers' data in order of appearance
material_names_ordered = [layer[0] for layer in layers]

# Flatten the materials dictionary in the order of appearance
material_features = []
for material_key in material_names_ordered:
    if material_key in materials:
        material_features.extend(materials[material_key])

# Flatten the layers list
layer_thicknesses = [thickness for _, thickness in layers]

# Combine all features into a single array
all_features = np.concatenate((material_features, layer_thicknesses, [qwi_target_shift], [max_applied_electric_field]))

input_tensor = torch.tensor(all_features, dtype=torch.float32)
print(input_tensor)
# Pass input data through the model
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    output_tensor = model(input_tensor)

# Get the predicted values
predicted_values = output_tensor.numpy()

print("Predicted values:", predicted_values)


tensor([2.3144e-01, 1.0037e+00, 5.9350e-02, 6.8082e-02, 5.1344e-01, 3.5225e+00,
        3.1648e-01, 7.5637e-01, 4.2259e-02, 5.2694e-02, 4.5853e-01, 3.5426e+00,
        2.3144e-01, 1.0037e+00, 5.9350e-02, 6.8082e-02, 5.1344e-01, 3.5225e+00,
        9.7757e+01, 9.6556e+01, 9.7757e+01, 3.0000e+01, 1.0000e+01])
Predicted values: [1.5535699e-03 1.1404514e-01 2.3015819e+00 1.4336526e-01]


In [69]:
def objective_function(heterostructure):
    input_tensor = torch.tensor(heterostructure, dtype=torch.float32)
    with torch.no_grad():
        model.eval()  
        output_tensor = model(input_tensor)
    return output_tensor.numpy()

# Define the search space (ranges for each parameter)
search_space = {
    'material_1_thickness': (0.1, 1.0),
    'material_2_thickness': (0.1, 1.0),
    # Add more parameters as needed
}

# Perform grid search
best_fom = -float('inf')
best_heterostructure = None

for material_1_thickness in np.linspace(search_space['material_1_thickness'][0], search_space['material_1_thickness'][1], num=10):
    for material_2_thickness in np.linspace(search_space['material_2_thickness'][0], search_space['material_2_thickness'][1], num=10):
        # Create heterostructure
        heterostructure = [material_1_thickness, material_2_thickness]  # Add more parameters as needed
        
        # Evaluate FOM
        fom = objective_function(heterostructure)
        
        # Update best FOM and heterostructure if necessary
        if fom > best_fom:
            best_fom = fom
            best_heterostructure = heterostructure

print("Best FOM:", best_fom)
print("Best Heterostructure:", best_heterostructure)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2 and 23x128)